# Mount

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Folder Name!

In [3]:
folder_name = "train_gdemo"

In [4]:
data_directory = 'drive/MyDrive/Colab Notebooks/Polyamanita/data/' + folder_name + '/'
model_directory = 'drive/MyDrive/Colab Notebooks/Polyamanita/models/' + 'FINAL' # + folder_name + '_2/'

model_file = model_directory + '/savedmodel.h5'
lite_model_file = model_directory + '/mushroom.tflite'

# Downloading libraries / imports

In [5]:
!pip -q install tensorflow keras
!pip -q install tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 24.2 MB/s eta 0:00:00


In [6]:
import os
import re
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import load_model # MAY NEED TO BE REPLACED W/ FUNCTIONAL API
import numpy as np
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

# Prevent OOM error

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True) # /OOM error

# Generate the label file for metadata

In [8]:
model = tf.keras.models.load_model(model_file)

In [9]:
classes = os.listdir(data_directory)
# print(classes)

prefix = '[0-9]+_'
label_string = ''

for i in range(len(classes)):
    s = classes[i]

    s = re.sub(prefix, '', s)
    s = re.sub('_', ' ', s)

    label_string += s + '\n'

label_string = label_string[:-1]
print(label_string)

Lycoperdon marginatum
Pleurotus pulmonarius
Clathrus columnatus
Scleroderma citrinum
Trametes lactinea
Omphalotus subilludens
Leucocoprinus fragilissimus
Phallus rugulosus
Chlorophyllum molybdites
Pycnoporus cinnabarinus
Leucocoprinus birnbaumii
Coprinopsis lagopus
Hortiboletus rubellus
Coprinellus disseminatus
Pholiota adiposa
Pleurotus ostreatus
Pluteus cervinus
Plicatura crispa
Xerocomellus chrysenteron
Tricholoma scalpturatum


In [10]:
# Generating a label file for output mapping
label_file_path = model_directory + 'label_file.txt'

fp = open(label_file_path, "w")
fp.write(label_string)
fp.close()

# Create Model Metadata

In [11]:
# Build model information
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "Polyamanita 10-Species Image Classifier"
model_meta.description = ("Identifies the species of mushroom in the center of the image.")
model_meta.version = "10spec"
model_meta.author = "Polyamanita"

In [12]:
# Create input metadata
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image (*.jpeg)"
input_meta.description = ("Input image to be classified. The expected image is 512 x 512, with three channels (red, blue, and green) per pixel. Each value in the tensor is a single byte between 0 and 255.")

input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (_metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (_metadata_fb.ContentProperties.ImageProperties)

input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (_metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]

input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

In [13]:
# Create output metadata
output_meta = _metadata_fb.TensorMetadataT()

output_meta.name = "probability"
output_meta.description = "Probabilities of the class labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (_metadata_fb.ContentProperties.FeatureProperties)

output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats

label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(label_file_path)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS

output_meta.associatedFiles = [label_file]

In [14]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(model_meta.Pack(b), _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

# Create TFLite file

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [16]:
tflite_model = converter.convert()

In [17]:
open(lite_model_file, "wb").write(tflite_model)

8967624

# Pack in the metadata

In [18]:
populator = _metadata.MetadataPopulator.with_model_file(lite_model_file)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([label_file_path])
populator.populate()

In [19]:
displayer = _metadata.MetadataDisplayer.with_model_file(lite_model_file)
# meta_json = os.path.join(model_directory + ".json")
json = displayer.get_metadata_json()

# print(displayer.get_packed_associated_file_list())
print(json)
# Optional: write out the metadata as a json file
# with open(export_json_file, "w") as f:
#   f.write(json_file)

{
  "name": "Polyamanita 10-Species Image Classifier",
  "description": "Identifies the species of mushroom in the center of the image.",
  "version": "10spec",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image (*.jpeg)",
          "description": "Input image to be classified. The expected image is 512 x 512, with three channels (red, blue, and green) per pixel. Each value in the tensor is a single byte between 0 and 255.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [

# Test the saved TFLite file

### urls

In [20]:
img_url = [
    
    ["40392_Armillaria_lutea","http://www.freenatureimages.eu/plants/Funghi,%20Paddestoelen,%20Mushrooms-Toadstools/Armillaria%20lutea/Armillaria%20lutea%201%2C%20Saxifraga-Jan%20de%20Laat.jpg"],
    ["40392_Armillaria_lutea","https://www.asergeev.com/pictures/archives/2013/1261/jpeg/14b.jpg"],
    ["40392_Armillaria_lutea","https://www.asergeev.com/pictures/archives/2016/1941/jpeg/08b.jpg"],

    ["40985_Byssomerulius_corium","https://btweenblinks.files.wordpress.com/2013/12/p1030106.jpg"],
    ["40985_Byssomerulius_corium","https://api.ispotnature.org/storage/app/sites/default/files/images/38683/31d25304cecb8a8de735fbe22854d0f2.jpg"],
    ["40985_Byssomerulius_corium","https://btweenblinks.files.wordpress.com/2013/12/p1030085.jpg"],

    ["61207_Coprinellus_micaceus","https://www.messiah.edu/Oakes/fungi_on_wood/gilled%20fungi/images/Cop%20micaceus%20DW.jpg"],
    ["61207_Coprinellus_micaceus","https://upload.wikimedia.org/wikipedia/commons/c/c3/Coprinellus_micaceus_Glimmer-Tintling.jpg"],
    ["61207_Coprinellus_micaceus","https://live.staticflickr.com/2947/33650872392_99a1e74718_b.jpg"],
    
    ["12919_Cylindrobasidium_laeve","https://live.staticflickr.com/65535/49484537306_b3f48b4039_b.jpg"],
    ["12919_Cylindrobasidium_laeve","https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2i5lvJBXSqveM-vaaE69DY2AhZId9kyAosQ&usqp=CAU"],
    ["12919_Cylindrobasidium_laeve","https://live.staticflickr.com/7851/45859587244_bd3d8fae5e_b.jpg"],

    ["14064_Fomitopsis_pinicola","https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Fomitopsis_pinicola_1.JPG/1200px-Fomitopsis_pinicola_1.JPG"],
    ["14064_Fomitopsis_pinicola","https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Fomitopsis_pinicola_109142.jpg/1200px-Fomitopsis_pinicola_109142.jpg"],
    ["14064_Fomitopsis_pinicola","https://www.mykoweb.com/CAF/photos/large/Fomitopsis_pinicola%28mgw-02%29.jpg"],

    ["14160_Ganoderma_pfeifferi", "https://www.first-nature.com/fungi/images/ganodermataceae/ganoderma-pfeifferi2.jpg"],
    ["14160_Ganoderma_pfeifferi", "https://www.first-nature.com/fungi/images/ganodermataceae/ganoderma-pfeifferi1.jpg"],
    ["14160_Ganoderma_pfeifferi", "https://c8.alamy.com/comp/DAYX2E/beeswax-bracket-fungus-ganoderma-pfeifferi-ganodermataceae-syn-ganoderma-DAYX2E.jpg"],

    ["17233_Mycena_galericulata","https://www.first-nature.com/fungi/images/mycenaceae/mycena-galericulata1.jpg"],
    ["17233_Mycena_galericulata","https://www.mushroomexpert.com/images/kuo7/mycena_galericulata_03.jpg"],
    ["17233_Mycena_galericulata","https://www.first-nature.com/fungi/images/mycenaceae/mycena-galericulata1.jpg"],

    ["20983_Trametes_versicolor","https://upload.wikimedia.org/wikipedia/commons/a/a3/Trametes_versicolor_G4_%281%29.JPG"],
    ["20983_Trametes_versicolor","https://ars.els-cdn.com/content/image/1-s2.0-S1877117319300353-f15-01-9780128177402.jpg"],
    ["20983_Trametes_versicolor","https://www.messiah.edu/Oakes/fungi_on_wood/poroid%20fungi/images/Tra%20versicolor%20Nathan%20Wilson.jpg"],

    ["21143_Tricholoma_scalpturatum","https://upload.wikimedia.org/wikipedia/commons/7/78/Tricholoma_scalpuratum_20061014wa.jpg"],
    ["21143_Tricholoma_scalpturatum","https://c8.alamy.com/zooms/9/1ac2cda5571c480ca5441eab637130cd/2dk86n8.jpg"],
    ["21143_Tricholoma_scalpturatum","https://www.hlasek.com/foto/tricholoma_scalpturatum_be4903.jpg"],

    ["11082_Xerocomellus_chrysenteron","https://www.mycodb.fr/photos/Xerocomellus_chrysenteron_2014_rp_1.jpg"],
    ["11082_Xerocomellus_chrysenteron","https://www.mykoweb.com/CAF/photos/large/Xerocomellus_diffractus_mgw-02.jpg"],
    ["11082_Xerocomellus_chrysenteron","https://www.mushroomexpert.com/images/kuo5/xerocomellus_chrysenteron_02.jpg"],

]

### test : TODO

In [21]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=lite_model_file)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [22]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

In [23]:
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[ -3.219892     9.565427    -7.930286    -4.3956285   -6.796214
   -3.5211556   -3.4429576    5.016625    -6.663361     0.99126035
    3.5936112    2.0306845   -9.737074     0.15841566  -6.353436
  -12.044646    -6.8551965    2.2383227   -8.002895     2.6204867 ]]
